In [10]:
import numpy as np
import scipy.stats as stats

from numpy.fft import fft, fftshift, ifftshift
from scipy.interpolate import interp1d
from time import time

In [11]:
N_SIMS = 20000                           # FIXME: number of simulations
N_BLOCKS = 100
N_STEPS = 250                            # FIXME: number of steps (excluding t = 0)
T = 1                                    # FIXME: time horizon
S0 = 1                                   # initial stock price (not using here for simplicity)

dt = T / N_STEPS                         # time interval

# GBM parameters
K = 1.1
R = 0.05                                 # FIXME: risk-free interest rate
Q = 0.02
MU = R - Q                               # FIXME: drift coefficient (GBM)
SIGMA = 0.1                              # FIXME: diffusion coefficient

MU = MU - 0.5 * SIGMA**2                 # fix drift coefficient for ABM

# Jump parameters
LAMBDA = 3                             # FIXME: rate of jump arrival, controls level of excess kurtosis
P = 0.3                                  # FIXME: probability of positive jump
ETA1 = 40                                 # FIXME: parameter of positive jump size
ETA2 = 12                                 # FIXME: parameter of negative jump size

dt = T / N_STEPS                         # time step
t_steps = np.linspace(0, T, N_STEPS + 1) # vector of time points

# Fourier parameters ---------------------------------------------------------------------------------------------
X_WIDTH = 6                                # TODO: Width of the interval
N_GRIDS = 2**8                             # TODO: Number of grid points
ALPHA = -10                                # TODO: Dampening factor for a call

In [12]:
# # ANALYTICAL SOLUTION ============================================================================================
# temp = time()

# # Black-Scholes-Merton formula
# d1 = (np.log(S0 / K) + (R - Q + 0.5 * SIGMA**2) * T) / (SIGMA * np.sqrt(T))
# d2 = d1 - SIGMA * np.sqrt(T)

# # BSM formula for call option
# def bs_call(S0, K, R, Q, SIGMA, T):
#     d1 = (np.log(S0 / K) + (R - Q + 0.5 * SIGMA**2) * T) / (SIGMA * np.sqrt(T))
#     d2 = d1 - SIGMA * np.sqrt(T)
#     return S0 * np.exp(-Q * T) * stats.norm.cdf(d1) - K * np.exp(-R * T) * stats.norm.cdf(d2)

# def bs_put(S0, K, R, Q, SIGMA, T):
#     d1 = (np.log(S0 / K) + (R - Q + 0.5 * SIGMA**2) * T) / (SIGMA * np.sqrt(T))
#     d2 = d1 - SIGMA * np.sqrt(T)
#     return K * np.exp(-R * T) * stats.norm.cdf(-d2) - S0 * np.exp(-Q * T) * stats.norm.cdf(-d1)

# call_analytical = bs_call(S0, K, R, Q, SIGMA, T)
# put_analytical = bs_put(S0, K, R, Q, SIGMA, T)

# # ================================================================================================================
# t_analytical = time() - temp

In [13]:
def payoff(x, xi, ALPHA, K, L, U, C, theta):
    # Scale
    S = C * np.exp(x)

    # Payoff; see e.g. Green, Fusai, Abrahams 2010, Eq. (3.24)
    g = np.exp(ALPHA * x) * np.maximum(theta * (S - K), 0) * (S >= L) * (S <= U)

    # Analytical Fourier transform of the payoff
    l = np.log(L / C)  # lower log barrier
    k = np.log(K / C)  # log strike
    u = np.log(U / C)  # upper log barrier

    # Integration bounds
    if theta == 1:  # call
        a = max(l, k)
        b = u
    else:  # put
        a = min(k, u)
        b = l

    # Green, Fusai, Abrahams 2010 Eq. (3.26) with extension to put option
    xi2 = ALPHA + 1j * xi
    G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2) - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)

    # Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
    if ALPHA == 0:
        G[len(G) // 2] = C * (np.exp(b) - np.exp(a) - np.exp(k) * (b - a))
    elif ALPHA == -1:
        G[len(G) // 2] = C * (b - a + np.exp(k - b) - np.exp(k - a))

    return S, g, G

In [14]:
def charfunction(xi, parameters, flag=0):
    """
    Compute the characteristic function for different pricing problems.
    :param xi: Fourier space grid.
    :param parameters: Parameters for the distributions.
    :param flag: Flag for backward (0) or forward (1) characteristic function.
    :return: Characteristic function.
    """
    meancorrection = (parameters['rf'] - parameters['q']) * parameters['dt'] - \
                     np.log(charfunction0(-1j, parameters))
    F = np.exp(1j * meancorrection * xi) * charfunction0(xi, parameters)
    if flag == 0:
        F = np.conj(F)
    return F

def charfunction0(xi, parameters):
    """
    Compute the characteristic function for various distributions.
    :param xi: Fourier space grid.
    :param parameters: Parameters for the distributions.
    :return: Characteristic function.
    """
    distr_type = parameters['distr']
    
    if distr_type == 1:  # Normal
        m = parameters['m']
        s = parameters['s']
        F = np.exp(1j * xi * m - 0.5 * (s * xi)**2)

    elif distr_type == 2:  # NIG
        alpha = parameters['alpha']
        beta = parameters['beta']
        delta = parameters['delta']
        F = np.exp(-delta * (np.sqrt(alpha**2 - (beta + 1j * xi)**2) - np.sqrt(alpha**2 - beta**2)))

    elif distr_type == 3:  # VG
        theta = parameters['theta']
        s = parameters['s']
        nu = parameters['nu']
        F = (1 - 1j * xi * theta * nu + 0.5 * nu * (s * xi)**2)**(-1 / nu)

    elif distr_type == 4:  # Meixner
        alpha = parameters['alpha']
        beta = parameters['beta']
        delta = parameters['delta']
        F = (np.cos(beta / 2) / np.cosh((alpha * xi - 1j * beta) / 2))**(2 * delta)

    elif distr_type == 5:  # CGMY
        C = parameters['C']
        G = parameters['G']
        M = parameters['M']
        Y = parameters['Y']
        F = np.exp(C * np.gamma(-Y) * ((M - 1j * xi)**Y - M**Y + (G + 1j * xi)**Y - G**Y))

    elif distr_type == 6:  # Kou
        s = parameters['s']
        lambd = parameters['lambda']
        pigr = parameters['pigr']
        eta1 = parameters['eta1']
        eta2 = parameters['eta2']
        F = np.exp(-0.5 * (s * xi)**2 + lambd * ((1 - pigr) * eta2 / (eta2 + 1j * xi) + pigr * eta1 / (eta1 - 1j * xi) - 1))

    elif distr_type == 7:  # Merton
        s = parameters['s']
        alpha = parameters['alpha']
        lambd = parameters['lambda']
        delta = parameters['delta']
        # F = np.exp(-0.5 * (s * xi)**2 + lambd * (np.exp(1j * xi * alpha - 0.5 * (delta * xi)**2) - 1))
        F = np.exp( 1j * xi * (MU - 0.5 * SIGMA**2) - 0.5 * (s * xi)**2 + lambd * (np.exp(1j * xi * alpha - 0.5 * (delta * xi)**2) - 1))


    elif distr_type == 8:  # Levy alpha-stable
        alpha = parameters['alpha']
        beta = parameters['beta']
        gamm = parameters['gamm']
        m = parameters['m']
        c = parameters['c']
        F = np.exp(1j * xi * m - c * np.abs(gamm * xi)**alpha * (1 - 1j * beta * np.sign(xi) * np.tan(np.pi * alpha / 2)))

    else:
        raise ValueError('Invalid distribution number')

    return F

In [15]:
def kernel(ngrid, xmin, xmax, parameters, alpha=0, disc=1, flag=0):
    """
    Set up grids, compute the characteristic function, and apply discounting.

    Parameters:
    ngrid (int): Number of grid points.
    xmin (float): Minimum value of the grid in real space.
    xmax (float): Maximum value of the grid in real space.
    parameters (dict): Dictionary of parameters for the characteristic function.
    alpha (float): Shift parameter, especially for Feng-Linetsky and convolution.
    disc (int): Discount factor in the density (0 for no, 1 for yes).
    flag (int): Type of characteristic function (0 for backward, 1 for forward).

    Returns:
    tuple: Grid in real space (x), density (h), grid in Fourier space (xi), Fourier transform of density (H).
    """

    N = ngrid // 2
    dx = (xmax - xmin) / ngrid
    x = dx * np.arange(-N, N)
    dxi = 2 * np.pi / (xmax - xmin)
    xi = dxi * np.arange(-N, N)

    # Assuming charfunction is defined elsewhere
    H = charfunction(xi + 1j * alpha, parameters, flag)  # characteristic function

    if disc == 1:
        H *= np.exp(-parameters.rf * parameters.dt)  # apply discount

    h = np.real(fftshift(fft(ifftshift(H)))) / (xmax - xmin)  # discounted kernel

    return x, h, xi, H

In [16]:
def parameters(distr, T, dt, rf, q=None):
    param = {'distr': distr, 'T': T, 'dt': dt, 'rf': rf, 'q': 0 if q is None else q}

    if distr == 1:  # Normal
        m = 0  # mean
        s = 0.4  # standard deviation

        param.update({'m': m, 's': s * (dt ** 0.5), 'lambdam': 0, 'lambdap': 0})

    elif distr == 2:  # NIG
        alpha, beta, delta = 15, -5, 0.5
        param.update({'alpha': alpha, 'beta': beta, 'delta': delta * dt, 'lambdam': beta - alpha, 'lambdap': beta + alpha, 'FLc': delta, 'FLnu': 1})

    elif distr == 3:  # VG
        C, G, M = 4, 12, 18
        nu = 1 / C
        theta = (1 / M - 1 / G) * C
        s = ((2 * C) / (M * G)) ** 0.5

        param.update({'nu': nu / dt, 'theta': theta * dt, 's': s * (dt ** 0.5), 'lambdam': -M, 'lambdap': G})

    elif distr == 4:  # Meixner
        alpha, beta, delta = 0.3977, -1.4940, 0.3462
        param.update({'alpha': alpha, 'beta': beta, 'delta': delta * dt})

    elif distr == 5:  # CGMY
        C, G, M, Y = 4, 50, 60, 0.7
        param.update({'C': C * dt, 'G': G, 'M': M, 'Y': Y, 'lambdam': -M, 'lambdap': G, 'FLc': 2 * C * abs(np.gamma(-Y) * np.cos(np.pi * Y / 2)), 'FLnu': Y})

    elif distr == 6:  # Kou double exponential
        s, lambda_, pigr, eta1, eta2 = SIGMA, LAMBDA, P, ETA1, ETA2
        param.update({'s': s * (dt ** 0.5), 'lambda': lambda_ * dt, 'pigr': pigr, 'eta1': eta1, 'eta2': eta2, 'lambdam': -eta1, 'lambdap': eta2, 'FLc': (s ** 2) / 2, 'FLnu': 2})

    elif distr == 7:  # Merton jump-diffusion
        s, alpha, lambda_, delta = SIGMA, MU_J, LAMBDA, SIGMA_J
        param.update({'s': s * (dt ** 0.5), 'alpha': alpha, 'lambda': lambda_ * dt, 'delta': delta, 'lambdam': 0, 'lambdap': 0, 'FLc': (s ** 2) / 2, 'FLnu': 2})

    elif distr == 8:  # Stable
        alpha, beta, gamm, m, c = 2, 0, 0.3 / (2 ** 0.5), 0, 1 / ((1 + (beta * np.tan(alpha / 2 * np.pi)) ** 2) ** 0.5)
        param.update({'alpha': alpha, 'beta': beta, 'gamm': gamm * dt, 'm': m, 'c': c})

    else:
        raise Exception('Invalid distribution type')

    return param


In [17]:
# # FOURIER SOLUTION ===============================================================================================
temp = time()

# Grids in real and Fourier space
N = N_GRIDS // 2
b = X_WIDTH / 2  # upper bound of the support in real space
dx = X_WIDTH / N_GRIDS  # grid step in real space
x = dx * np.arange(-N, N)  # grid in real space
dxi = np.pi / b  # Nyquist relation: grid step in Fourier space
xi = dxi * np.arange(-N, N)  # grid in Fourier space

# FOR KJD
param = parameters(distr = 6, T = T, dt = T, rf = R, q = Q)
[x, fc, xi, psi_call] = kernel(N_GRIDS, -b, b, param, ALPHA, 0, 1)
[x, fp, xi, psi_put] = kernel(N_GRIDS, -b, b, param, -ALPHA, 0, 1)

# Fourier transform of the payoff
b = X_WIDTH / 2  # upper bound of the support in real space
U = S0 * np.exp(b)
L = S0 * np.exp(-b)
_, gc, Gc = payoff(x, xi, ALPHA, K, L, U, S0, 1)  # call
S, gp, Gp = payoff(x, xi, -ALPHA, K, L, U, S0, 0)  # put

# Discounted expected payoff computed with the Plancherel theorem
c = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gc * np.conj(psi_call))))) / X_WIDTH  # call
call_fourier = interp1d(S, c, kind='cubic')(S0)
p = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gp * np.conj(psi_put))))) / X_WIDTH  # put
put_fourier = interp1d(S, p, kind='cubic')(S0)

# interp1d(S, p, kind='cubic')(S0) means that we interpolate the function p(S) at the point S0, S is the grid of the function p(S), S0 is in S

# # ================================================================================================================
t_fourier = time() - temp

In [18]:
# MONTE CARLO SOLUTION ===========================================================================================
temp = time()

call_mc_blocks = np.zeros(N_BLOCKS)
put_mc_blocks = np.zeros(N_BLOCKS)

# Monte Carlo simulation via KJD
for i in range(N_BLOCKS):

    N = np.random.poisson(LAMBDA * dt, (N_STEPS, N_SIMS))

    # Generate Bilateral Exponential R.V.       
    U = np.random.uniform(size = (N_STEPS, N_SIMS))
    J = np.where(U >= 1 - P, -1 / ETA1 * np.log((1 - U) / P), 1 / ETA2 * np.log(U / (1 - P)) * (U < 1 - P))
    J *= N

    # Add jump increments to ABM increments    
    S = MU * dt + SIGMA * np.sqrt(dt) * np.random.randn(N_STEPS, N_SIMS) + J
    S = np.vstack([np.zeros(N_SIMS), np.cumsum(S, axis=0)])
    S += S0

    call_mc_blocks[i] = np.exp(-R * T) * np.maximum(S - K, 0).mean()
    put_mc_blocks[i] = np.exp(-R * T) * np.maximum(K - S, 0).mean()



    
# Obtaining the results
call_mc = call_mc_blocks.mean()
put_mc = put_mc_blocks.mean()

# Standard error
call_mc_se = np.sqrt(call_mc_blocks.var() / N_BLOCKS)
put_mc_se = np.sqrt(put_mc_blocks.var() / N_BLOCKS)

# ================================================================================================================
t_mc = time() - temp

In [19]:
# PRINT RESULTS ===================================================================================================
print(f"{'':18s}{'Call':15s}{'Put':15s}{'CPU Time/s':15s}{'Operations':15s}")
# print(f"{'BS Analytical':15s}{call_analytical:15.10f}{put_analytical:15.10f}{t_analytical:15.10f}{1:13d}")
print(f"{'Fourier':15s}{call_fourier:15.10f}{put_fourier:15.10f}{t_fourier:15.10f}{N_GRIDS:13d}")
print(f"{'Monte Carlo':15s}{call_mc:15.10f}{put_mc:15.10f}{t_mc:15.10f}{N_BLOCKS * N_SIMS:13d}")
print(f"{'MC S.E.':15s}{call_mc_se:15.10f}{put_mc_se:15.10f}")

                  Call           Put            CPU Time/s     Operations     
Fourier           0.0432285053  41.6791391697   0.0077831745          256
Monte Carlo       0.0028151509   0.1585246098  58.0463194847      2000000
MC S.E.           0.0000076876   0.0000731161


In [20]:
# Verify put-call parity
call_fourier - put_fourier - S0 + K * np.exp(-R * T)

-41.58955829744969